# Data Scrubbing

#### 1) Importing libraries and functions.

In [1]:
import pandas as pd
import numpy as np
import Functions
from Functions import remove_dupes, percent_null_df, percent_null_col, df_snapshot, find_nans, determine_dtype, make_ints, show_dupes

#### 2) Reading in the .csv with pandas. Creating a copy to avoid error messages.

In [2]:
x = pd.read_csv('https://raw.githubusercontent.com/snepaul179/Real_EstateModel-onl01-dtsc-pt-052620/master/kc_house_data.csv')
data = x.copy()

#### 3) Examining the first few rows.

In [3]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [4]:
# in order to properly sort by date, I want to change 'date' to datetime object

In [5]:
data['date'] =  pd.to_datetime(data.date, infer_datetime_format=True)  

In [6]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [7]:
# later in the process it was brought up that it would be possible for the same house to be sold twice
# i rechecked the dataframe, and found that there were some instances of this.
# i am going to investigate the duplicates further.

In [8]:
# step one: sorting by id and date reveals that there are id numbers that have more than one sale
data.sort_values(by=['id', 'date'])

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2494,1000102,2014-09-16,280000.0,6,3.00,2400,9373,2.0,NaN,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
2495,1000102,2015-04-22,300000.0,6,3.00,2400,9373,2.0,0.0,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
6729,1200019,2014-05-08,647500.0,4,1.75,2060,26036,1.0,NaN,0.0,...,8,1160,900.0,1947,0.0,98166,47.4444,-122.351,2590,21891
8404,1200021,2014-08-11,400000.0,3,1.00,1460,43000,1.0,0.0,0.0,...,7,1460,0.0,1952,0.0,98166,47.4434,-122.347,2250,20023
8800,2800031,2015-04-01,235000.0,3,1.00,1430,7599,1.5,0.0,0.0,...,6,1010,420.0,1930,0.0,98168,47.4783,-122.265,1290,10320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16723,9842300095,2014-07-25,365000.0,5,2.00,1600,4168,1.5,0.0,0.0,...,7,1600,0.0,1927,0.0,98126,47.5297,-122.381,1190,4168
3257,9842300485,2015-03-11,380000.0,2,1.00,1040,7372,1.0,0.0,0.0,...,7,840,200.0,1939,0.0,98126,47.5285,-122.378,1930,5150
7614,9842300540,2014-06-24,339000.0,3,1.00,1100,4128,1.0,0.0,0.0,...,7,720,380.0,1942,NaN,98126,47.5296,-122.379,1510,4538
20963,9895000040,2014-07-03,399900.0,2,1.75,1410,1005,1.5,0.0,0.0,...,9,900,510.0,2011,0.0,98027,47.5446,-122.018,1440,1188


In [9]:
# step two: creating a subset of data that I think will be useful for comparing changes in price over time in the same area
idents = data[data.id.duplicated(keep=False)]

In [10]:
idents.shape # 353 rows where identity is not unique

(353, 21)

In [11]:
idents.sort_values(by=['id', 'date']) # for the most part seems like there are different dates, will have to check more later.

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2494,1000102,2014-09-16,280000.0,6,3.00,2400,9373,2.0,NaN,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
2495,1000102,2015-04-22,300000.0,6,3.00,2400,9373,2.0,0.0,0.0,...,7,2400,0.0,1991,0.0,98002,47.3262,-122.214,2060,7316
16800,7200179,2014-10-16,150000.0,2,1.00,840,12750,1.0,0.0,0.0,...,6,840,0.0,1925,0.0,98055,47.4840,-122.211,1480,6969
16801,7200179,2015-04-24,175000.0,2,1.00,840,12750,1.0,0.0,0.0,...,6,840,0.0,1925,NaN,98055,47.4840,-122.211,1480,6969
11421,109200390,2014-08-20,245000.0,3,1.75,1480,3900,1.0,0.0,0.0,...,7,1480,0.0,1980,0.0,98023,47.2977,-122.367,1830,6956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6340,9828200460,2015-01-06,430000.0,2,1.00,700,4800,1.0,0.0,0.0,...,7,700,0.0,1922,0.0,98122,47.6147,-122.300,1440,4800
15185,9834200305,2014-07-16,350000.0,3,1.00,1790,3876,1.5,0.0,0.0,...,7,1090,?,1904,0.0,98144,47.5750,-122.288,1360,4080
15186,9834200305,2015-02-10,615000.0,3,1.00,1790,3876,1.5,0.0,0.0,...,7,1090,700.0,1904,0.0,98144,47.5750,-122.288,1360,4080
1084,9834200885,2014-07-17,360000.0,4,2.50,2080,4080,1.0,0.0,0.0,...,7,1040,1040.0,1962,0.0,98144,47.5720,-122.290,1340,4080


In [12]:
cols = ['id','date','price','sqft_lot','zipcode'] # trimming this down to just the factors i am curious about for analysis

In [13]:
idents = idents[cols]

In [14]:
idents['id'].value_counts() # checking to get an idea of how many times each one is duplicated, for the most part only once

795000620     3
8651402750    2
5536100020    2
9238500040    2
1995200200    2
             ..
8062900070    2
4139440480    2
2568300045    2
2143700830    2
2023049218    2
Name: id, Length: 176, dtype: int64

In [15]:
idents.to_csv('idents.csv') # save the idents.csv for later use, might be good to compare price per squarefoot over time

#### 4) Next, I want to set the index to id.

In [16]:
data.set_index('id', inplace=True)

#### 5) Now, I will run the function find_nans, to identify the columns with nan values, and return the number of nans as well as a list of the unique values, and number of unique values.

In [17]:
find_nans(data)

waterfront has  2376 NaN values.
waterfront has  3 unique values.
[nan, 0.0, 1.0] 

view has  63 NaN values.
view has  6 unique values.
[0.0, nan, 1.0, 2.0, 3.0, 4.0] 

yr_renovated has  3842 NaN values.
yr_renovated has  71 unique values.
[0.0, 1945.0, 1954.0, 1970.0, 1971.0, 1980.0, 1991.0, nan, 1934.0, 1940.0, 1944.0, 1946.0, 1948.0, 1950.0, 1951.0, 1953.0, 1955.0, 1956.0, 1957.0, 1958.0, 1959.0, 1960.0, 1962.0, 1963.0, 1964.0, 1965.0, 1967.0, 1968.0, 1969.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0, 1977.0, 1978.0, 1979.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0, 1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1992.0, 1993.0, 1994.0, 1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0] 



#### 6) In the 'waterfront' column, 2000+ values are missing. 
- A) I will segment the dataframe to exclude 'waterfront' because the data is unavailable. 
- B) I will make another dataframe that includes only values for which waterfront data is included. 
- C) I suspect that if we were to inspect the data by zip code, it would be present only in certain coastal zipcodes and not in landlocked zip codes. I think there is potential to do a nice visualization with a choropleth map, as well as examining the features of each zipcode further.

In [18]:
wf_data = data[data.waterfront.notna()]
no_wf_data = data[data.waterfront.notna()]
wf_data.to_csv('wf_data.csv')
no_wf_data.to_csv('no_wf_data.csv')

data = data.drop(columns=['waterfront'])

#### 8) 'veiw' only has 63 NaNs, so we can probably eliminate those rows from the data. 
- A) But just for fun, I'll keep a dataframe with view data in tact in case we are working on an analysis in which view is unimportant, and can be dropped.

In [19]:
data_wo_view = data.drop('view', axis=1)
data = data[data.view.notna()]

#### 9) Need to look further into 'yr_renovated.'
- A) 'yr_renovated' has a few values that are strange. There are some that have a 0, which I believe indicates they have never been renovated. I'm curious as to why some are NaNs and some are zeroes. I'm going to test and see if this has something do with the age of the home.

In [20]:
yr_reno_nans = data[data.yr_renovated.isna()]
print(yr_reno_nans.yr_built.min())
print(yr_reno_nans.yr_built.max())

1900
2015


- B) There does not seem to be a relationship with the age, as all ages seem to be represented. Now, I'll take a look at the subset of the data where yr_renovated is 0, and non NaN.

In [21]:
year_reno_zeroes = data[data.yr_renovated == 0.0]
print(year_reno_zeroes.yr_built.min())
print(year_reno_zeroes.yr_built.max())

1900
2015


- C) Perhaps there is a relationship with zipcode?

In [22]:
yr_reno_nans = data[data.yr_renovated.isna()]
print(len(yr_reno_nans.zipcode.unique()))

year_reno_zeroes = data[data.yr_renovated == 0.0]
print(len(year_reno_zeroes.zipcode.unique()))

70
70


- D) I don't see a relationship between them. So for now, I will create a subset of the data that excludes NaN values, in case we need to analyze a relationship between renovations and price. 

In [23]:
data['yr_renovated'] = data.yr_renovated.fillna(0.0)

#### 10) Next, I'm going to use a function called determine_dtype to print an organized of what kind of data types we are dealing with, so we can make decisions about data types that should be changed.

In [25]:
determine_dtype(data)

Objects: 
 ['sqft_basement'] 

Integers: 
 ['bedrooms', 'sqft_living', 'sqft_lot', 'condition', 'grade', 'sqft_above', 'yr_built', 'zipcode', 'sqft_living15', 'sqft_lot15'] 

Floats:
 ['price', 'bathrooms', 'floors', 'view', 'yr_renovated', 'lat', 'long'] 



- A) I only need to change data types if something is an object that shouldn't be an object, or a float when it doesn't need to be. I take a look at head one more time to get a since of what the values look like.

In [26]:
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,
7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,3,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,3,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,3,6,770,0.0,1933,0.0,98028,47.7379,-122.233,2720,8062
2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,5,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,3,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


- B) It looks like sqft_basement shouldn't be an object, and it would probably be more readable if price, view, and yr_renovated were integers instead of floats. I can use one function, make_ints, to make both objects and floats integers, by feeding it the list of columns I want to change.

In [27]:
cols = ['price', 'view', 'sqft_basement', 'yr_renovated']
make_ints(data,cols)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,
7129300520,2014-10-13,221900,3,1.00,1180,5650,1.0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,2014-12-09,538000,3,2.25,2570,7242,2.0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,2015-02-25,180000,2,1.00,770,10000,1.0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,2014-12-09,604000,4,3.00,1960,5000,1.0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,2015-02-18,510000,3,2.00,1680,8080,1.0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,2014-05-21,360000,3,2.50,1530,1131,3.0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
6600060120,2015-02-23,400000,4,2.50,2310,5813,2.0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
1523300141,2014-06-23,402101,2,0.75,1020,1350,2.0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007


#### 11) Now that the data is cleaned up a bit, I want to create a few more features. 
- A) I will use 'basement' to indicate 0 for no basement and 1 for basement, 
- B) 'reno' to indicate 0 for renovation and 1 for no renovation. 
- C) I will also include another column called 'yr_sold' to isolate the year sold as an integer so I can create another column called 'age_at_sale' that indicate the age of the home on the sale date.

In [28]:
data['basement'] = data.sqft_basement.apply(lambda x: 1 if x > 0 else 0)
data['reno'] = data.yr_renovated.apply(lambda x: 1 if x > 0 else 0)
data['yr_sold'] = pd.DatetimeIndex(data['date']).year
data['age_at_sale'] = data.yr_sold-data.yr_built

14) I will arrange the columns in a more readable way by putting like-items near each other.

In [29]:
x = ['price', 'date', 'yr_sold', 'yr_built', 'age_at_sale', 'reno', 'yr_renovated', 'condition', 'grade', 'view', 'floors', 'bedrooms', 'bathrooms', 'basement', 'sqft_lot', 'sqft_living', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'lat', 'long', 'zipcode']
data = data.reindex(columns=x)

In [30]:
data.head()

,price,date,yr_sold,yr_built,age_at_sale,reno,yr_renovated,condition,grade,view,...,basement,sqft_lot,sqft_living,sqft_basement,sqft_above,sqft_living15,sqft_lot15,lat,long,zipcode
id,,,,,,,,,,,,,,,,,,,,,
7129300520,221900,2014-10-13,2014,1955,59,0,0,3,7,0,...,0,5650,1180,0,1180,1340,5650,47.5112,-122.257,98178
6414100192,538000,2014-12-09,2014,1951,63,1,1991,3,7,0,...,1,7242,2570,400,2170,1690,7639,47.7210,-122.319,98125
5631500400,180000,2015-02-25,2015,1933,82,0,0,3,6,0,...,0,10000,770,0,770,2720,8062,47.7379,-122.233,98028
2487200875,604000,2014-12-09,2014,1965,49,0,0,5,7,0,...,1,5000,1960,910,1050,1360,5000,47.5208,-122.393,98136
1954400510,510000,2015-02-18,2015,1987,28,0,0,3,8,0,...,0,8080,1680,0,1680,1800,7503,47.6168,-122.045,98074


In [31]:
data.to_csv('kc_re_data.csv')

In [ ]:
# what year had a market boom? (plot)